In [1]:
import os
import pprint
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon


In [2]:
def getCutPoint(filePath):
    ''' 
    open cutpoint txt file
    the set the list to a,b,d,c for create shapefile
    '''
    returnList = []
    coorList = []
    with open(filePath, "r") as file:
        lines = file.readlines()

    pointCount = 0
    eachShapePoint = []
    for eachPoint in lines:
        eachPoint = eachPoint.strip()
        if not eachPoint:
            continue
        latlon = eachPoint.split(",")
        eachShapePoint.append((float(latlon[0]), float(latlon[1])))
        pointCount += 1

        if pointCount == 4:
            coorList.append(eachShapePoint)
            pointCount = 0 
            eachShapePoint = []

    for a, b, c, d in coorList:
        returnList.append([a, b, d, c])

    return returnList


def createShapeFile(cutPointList, outputShapefile):
    """
    Convert a list of 4-point (lat, lon) polygons into a shapefile.
    
    Parameters:
    - cutPointList: List of polygons, where each polygon is a list of 4 (lat, lon) tuples.
    - outputShapefile: Path to save the output shapefile (.shp).
    """
    rectangles = []
    ids = []

    for idx, coords in enumerate(cutPointList):
        # Convert (lat, lon) to (lon, lat)
        coords = [(lon, lat) for lat, lon in coords]

        # Close the polygon
        coords.append(coords[0])

        # Create Polygon
        rect = Polygon(coords)
        rectangles.append(rect)
        ids.append(idx + 1)

    # Create GeoDataFrame with EPSG:4326 (WGS84)
    gdf = gpd.GeoDataFrame({'id': ids, 'geometry': rectangles}, crs="EPSG:4326")

    # Save to shapefile
    gdf.to_file(outputShapefile)



In [3]:
cutPointList = getCutPoint("./2025CutPoint.txt")
createShapeFile(cutPointList, "./ShapeFile/ShapeFile2025.shp")